In [1]:
!rm -rf '/content/img_download'

### Install packages

In [2]:
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (86.0.4240.75-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


### Import libraries

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from urllib.request import urlretrieve
import numpy as np
import pickle as pkl
import time
import cv2
import os

### set option driver

In [4]:
chrome_op = Options()
chrome_op.add_argument('--headless')
chrome_op.add_argument('--no-sandbox')
chrome_op.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_op)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


### Get images url

In [5]:
# create a folder
!mkdir '/content/img_download'

In [6]:
# select the site
driver.get('https://www.google.com/')

# found the search bar and send the key
search_bar = driver.find_element_by_name('q')
search_bar.send_keys('picture of faces')
search_bar.submit()

# images page
img_page = driver.find_element_by_link_text('Images')
img_page.get_attribute('href')
img_page.click()

# scroll the pages
value = 0
for i in range(30):
  driver.execute_script(f'window.scrollTo(0, {value*i});')
  value += 100
  time.sleep(3)

# get images tag
elm = driver.find_element_by_id('islmp')
img_tag = elm.find_elements_by_tag_name("img")

# get images src
counter=0

for i, img in enumerate(img_tag):
  src = img.get_attribute('src')

  try:

    if src != None:
      src = str(src)
      urlretrieve(src, os.path.join('img_download', f'image_{i}.jpg'))
      counter += 1

  except:
    pass

print(f'download {counter} images from {len(img_tag)}')

download 421 images from 448


### detect the images with faces

In [7]:
casc_path = os.path.dirname(cv2.__file__) + '/data/haarcascade_frontalface_default.xml'
face_detect = cv2.CascadeClassifier(casc_path)

In [8]:
def image_to_array(dir, files):
  img_array = []
  counter = 0

  for file_ in files:
    try:
      image = cv2.imread(os.path.join(dir, file_))
      img_array.append(image)
    except:
      counter += 1
  if counter != 0:
    print(f"can't read {counter} images")
  return img_array

In [9]:
for dir, _, files in os.walk('/content/img_download'):
  image_array = image_to_array(dir, files)

In [10]:
face_list= []

for img in image_array:
  
  rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  faces = face_detect.detectMultiScale(rgb, 1.3, 5)

  for (x,y,w,h) in faces:
    roi = rgb[y:y+h, x:x+w]
    resize = cv2.resize(roi, (64,64))
    resize = resize/255
    face_list.append(resize)

print(f'found {len(face_list)} faces')

found 717 faces


In [11]:
face_array = np.array(face_list)

In [12]:
face_array.shape

(717, 64, 64, 3)

### dump the image array

In [19]:
pkl.dump(face_array, open('/content/drive/My Drive/Data Science/Project/Mask detection/dumps/without_mask.array', 'wb'))